In [1]:
# !pip3 install monai
# !pip3 install -U Setuptools
# !pip install torchvision
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install adabelief-pytorch==0.2.0

# !pip install ipywidgets widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
import json
import numpy as np
import PIL.Image
import PIL.ImageDraw
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import os
import cv2

# Define shape_to_mask function

參照老師

In [ ]:
def shape_to_mask(
    img_shape, points, shape_type=None, line_width=10, point_size=5
):
    mask = np.zeros(img_shape[:2], dtype=np.uint8) 
    mask = PIL.Image.fromarray(mask) 
    
    draw = PIL.ImageDraw.Draw(mask) 

    xy = [tuple(point) for point in points] 
    
    if shape_type == "circle":
        assert len(xy) == 2, "Shape of shape_type = circle must have 2 points"
        (cx, cy), (px, py) = xy
        d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
        draw.ellipse([cx - d, cy - d, cx + d, cy + d], outline=1, fill=1) 
    elif shape_type == "rectangle":
        assert len(xy) == 2, "Shape of shape_type=rectangle must have 2 points"
        draw.rectangle(xy, outline=1, fill=1)
    elif shape_type == "line":
        assert len(xy) == 2, "Shape of shape_type=line must have 2 points"
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "linestrip":
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "point":
        assert len(xy) == 1, "Shape of shape_type=point must have 1 points"
        cx, cy = xy[0]
        r = point_size
        draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=1, fill=1)
    else:
        assert len(xy) > 2, "Polygon must have points more than 2" 
        draw.polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask 

# Set process folder

In [2]:
# path
folder_path = "{YOUR PATH}"

# Create Visualize Function

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):  
        plt.subplot(1, n, i + 1)  
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

# Processing all json file in the folder

In [ ]:
 
os.makedirs(folder_path.replace("Train_Annotations", "Train_Annotations_png"))

for filename in os.listdir(folder_path):
    if 'json' in filename:
        # Read in all all the data from the CSV file      
        json_path = os.path.join(folder_path, filename)       
        
        write_msk_img_name = filename.replace("json","png")
        write_folder_path = folder_path.replace("Train_Annotations","Train_Annotations_png")
        
        img_path = folder_path.replace("/Train_Annotations/","/Train_Images/") + filename.replace("json","jpg")
        img = Image.open(img_path)

        #Read Json file
        with open(json_path, "r",encoding="utf-8") as f:
            dj = json.load(f)

        temp_mask_img = np.zeros([dj['imageHeight'], dj['imageWidth']],dtype=np.uint8)

        #Plot each mask into mask_img
        for i in range(len(dj['shapes'])):
            mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=dj['shapes'][i]['shape_type'],line_width=1, point_size=1)            
            temp_mask_img = temp_mask_img + mask.astype(int) 
        temp_mask_img = (temp_mask_img>0).astype(int)
        
        print(f"temp_mask_img.range is {temp_mask_img.max()} to {temp_mask_img.min()}")
        # Save the file
        cv2.imwrite(write_folder_path + write_msk_img_name, temp_mask_img*255)

# train

In [ ]:
import logging
import os
import sys
import time
import tempfile
import glob
import matplotlib.pyplot as plt
import numpy as np

import setuptools
import torch
import torchvision
from PIL import Image
from torch.utils.data import DataLoader 



import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference, SimpleInferer
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,  
    LoadImaged,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    AsChannelFirstd,
    Resized,
    # RandScaleCropd,
    RandRotated,
    SaveImage,
    # RandSpatialCropd,
    Resize,
)
from adabelief_pytorch import AdaBelief
import segmentation_models_pytorch as smp

# Monai config

In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
os.listdir('../SEG_Train_Datasets')

# Set Data folder

In [ ]:
# Set the Data folder
data_path = './SEG_Train_Datasets/'

In [ ]:
# Load train files
for_test = 256


tempdir = data_path + "Train_Images"
train_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "Train_Annotations_png"
train_segs = sorted(glob.glob(os.path.join(tempdir, "*.png")))


print(" {} train_images and {} train_segs".format(len(train_images[for_test:]), len(train_segs[for_test:])))


train_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[for_test:], train_segs[for_test:])]



# For validation 

In [ ]:
val_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[:for_test], train_segs[:for_test])]
print(" {} val_images and {} val_segs".format(len(train_images[:for_test]), len(train_segs[:for_test])))

# Define Trasform for image and segmentation

In [ ]:
width, length = 800, 800


# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        RandRotated(keys=["img", "seg"],range_x= 3.14),
        Resized(keys=["img", "seg"], spatial_size=[width, length]),
        RandRotate90d(keys=["img", "seg"], prob=0.3, spatial_axes=[0, 1]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        Resized(keys=["img"], spatial_size=[width, length]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)

# Check and visualize the transform results


In [ ]:
# define dataset, data loader
check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
print(type(check_ds))

# Create DataLoader for train and validation data

In [ ]:
# batch_size
batch_size = 16
# create a training data loader
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

train_loader = DataLoader(
    train_ds,
    batch_size= batch_size,
    shuffle=True,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)

# create a validation data loader
val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size= batch_size, collate_fn=list_data_collate)



# Define metric and post-processing

1. Monai有Dice的metrics與loss
2. define output transform (老師提供)


In [ ]:
dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])

# Set environment

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Create Visualize Function

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, cmap= 'gray')
    plt.show()

# Model Training

#### create UNet, DiceLoss and Adam optimizer

> 這部分由老師上課提供的範例程式做修改


In [ ]:
save_path = 'tf_efficientnetv2_m_in21ft1k_ver3'

In [ ]:
aux_params=dict(
    pooling='avg',           
    dropout=0.4,              
    activation=None,      
    classes=1,               
)

encodes = 'tu-tf_efficientnetv2_m_in21ft1k'
model = smp.DeepLabV3Plus(encodes, aux_params=aux_params)
model = torch.nn.DataParallel(model).to(device)

In [ ]:
optimizer = AdaBelief(model.parameters(), lr=1e-4, eps=1e-16, betas=(0.9, 0.98923), weight_decouple = True, rectify = False, weight_decay = 1e-4)
loss_function = monai.losses.DiceLoss(sigmoid=True)

In [ ]:
#### start a typical PyTorch training
total_epochs = 350
val_interval = 1
best_metric = 100   #best
best_metric_epoch = -1  #best
epoch_loss_values = list()   #empty list = []
metric_values = list()


for epoch in range(total_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{total_epochs}")

    # 開始訓練
    model.train()
    epoch_loss = 0
    step = 0

    for iters, batch_data in enumerate(train_loader):

        step += 1
        epoch_len = len(train_ds) // train_loader.batch_size   
        print(f" {step}/{epoch_len}" , end='\r')

        inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)

        optimizer.zero_grad()     
        outputs, label = model(inputs)    
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    local_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())    
    print("\n", f"{local_time} epoch {epoch + 1} training average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            steps = 0
            loss_val = 0

            for val_data in val_loader:
                val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                
#                 roi_size = (800, 800)
#                 sw_batch_size = 32
#                 # predict
#                 val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model) 
                
                val_outputs, val_out_label = model(val_images) #forward
                val_outputs = Resize([-1, 1716, 942])(val_outputs)

                val_loss = monai.losses.DiceLoss(sigmoid=True)(val_outputs, val_labels)
                loss_val += val_loss
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                

                # compute metric for current iteration
                if  steps < 3:
                    print("val loss", val_loss)
                    visualize( 
                        image=val_images[0].cpu().permute(1,2,0), 
                        ground_truth_mask=val_labels[0].cpu().permute(1,2,0), 
                        predicted_mask=val_outputs[0].cpu().permute(1,2,0)

                    )  
                steps += 1
            # aggregate the final mean dice result
            print("val_loss = ", loss_val / steps)
            val_loss_mean = loss_val / steps

            # reset the status for next validation round
            metric_values.append(val_loss_mean)

            if val_loss_mean < best_metric:
                best_metric = val_loss_mean
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), f"{save_path}.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current val mean dice loss: {:.4f} best val mean dice loss: {:.4f} at epoch {}".format(
                    epoch + 1, val_loss_mean, best_metric, best_metric_epoch
                )
            )

print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# 讀取模型

In [ ]:
model.load_state_dict(torch.load("{}.pth".format(save_path)))
# model.load_state_dict(torch.load("rou.pth"))

In [ ]:
model.eval()
dice_metric.reset()
with torch.no_grad():
    val_images = None
    val_labels = None
    val_outputs = None
    for val_data in val_loader:
        val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)

#         roi_size = (800, 800)
#         sw_batch_size = 4
#         val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model) 

        val_outputs, val_out_label = model(val_images) #forward
        val_outputs = Resize([-1, 1716, 942])(val_outputs)
        val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                
              
        # compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)
        print(dice_metric.aggregate())
    # aggregate the final mean dice result
    metric = dice_metric.aggregate().item()
    print("dice metric = ", metric)
    # reset the status for next validation round
    dice_metric.reset()

# 上傳結果

### public

In [ ]:
tempdir = "./Public_Image/"
test_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))


print(" {} test_images".format(len(test_images)))

test_files = [{"img": img} for img in test_images[:]]
test_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data=test_files, transform=test_transforms)

In [ ]:
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)

### 確認儲存路徑(public)

In [ ]:
pub_path = sorted(glob.glob(tempdir + "*.jpg"))
pub_path[0].split("/")[-1].split(".")[0]

In [ ]:
model.eval()
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)
        
        # define sliding window size and batch size for windows inference
#         roi_size = (800, 800)
#         sw_batch_size = 4
#         test_outputs = sliding_window_inference(test_images, roi_size, sw_batch_size, model)

        test_outputs, test_out_label = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)
        
        saverPD = SaveImage(output_dir=f"./{save_path}/output", output_ext=".png", output_postfix=f"{pub_path[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())

    dice_metric.reset()

### Private

In [ ]:
os.getcwd()

In [ ]:
tempdir = "./Pruvate_image/"
private_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))


print(f" {len(private_images)} private_images")

private_files = [{"img": img} for img in private_images[:]]
private_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data= private_files, transform=test_transforms)

In [ ]:
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)

In [ ]:
pri_path = sorted(glob.glob(tempdir + "*.jpg"))
pri_path[0].split("/")[-1].split(".")[0]

In [ ]:
model.eval()
steppp = 0
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)
        # define sliding window size and batch size for windows inference

        test_outputs, test_out_label = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)

        saverPD = SaveImage(output_dir=f"./{save_path}/output", output_ext=".png", output_postfix=f"{pri_path[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())
        
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]        

In [ ]:
predict = sorted(glob.glob(f"./{save_path}/output/*.png"))
predict[:5]

In [ ]:
for img in predict:
    os.rename(img, os.path.join(*img.split("/")[:-1], img.split("/")[-1].split("_", 1)[-1]))

# zip

> if need

In [ ]:
# !zip -r effnetv2.zip tf_efficientnetv2_m_in21ft1k_ver3/output